In [1]:
print(sc)

<SparkContext master=local[*] appName=PySparkShell>


In [2]:
sc.version

'3.0.1'

In [3]:
# lista de transformações e ações do Spark

# https://spark.apache.org/docs/latest/rdd-programming-guide.html

In [4]:
# Analisando dados do Uber
from pandas import read_csv

In [5]:
uberFile = read_csv("uber.csv")

In [6]:
type(uberFile)

pandas.core.frame.DataFrame

In [7]:
uberFile.head(5)

,dispatching_base_number,date,active_vehicles,trips
0,B02512,1/1/2015,190,1132
1,B02765,1/1/2015,225,1765
2,B02764,1/1/2015,3427,29421
3,B02682,1/1/2015,945,7679
4,B02617,1/1/2015,1228,9537


In [8]:
# transf. o dataframe pandas em dataframe spark
uberDF = sqlContext.createDataFrame(uberFile)

In [9]:
type(uberDF)

pyspark.sql.dataframe.DataFrame

In [10]:
# criando RDD a partir de arq .csv
uberRDD = sc.textFile("uber.csv")

In [11]:
type(uberRDD)

pyspark.rdd.RDD

In [12]:
uberRDD.count()

355

In [13]:
uberRDD.first()

'dispatching_base_number,date,active_vehicles,trips'

In [15]:
# dividindo o arq em colunas, separadas por ,
uberLinhas = uberRDD.map(lambda line: line.split(","))

In [16]:
type(uberLinhas)

pyspark.rdd.PipelinedRDD

In [17]:
# coletando valores distintos da primeira coluna
uberLinhas.map(lambda linha: linha[0]).distinct().count()

7

In [18]:
uberLinhas.map(lambda linha: linha[0]).distinct().collect()

['dispatching_base_number',
 'B02765',
 'B02682',
 'B02598',
 'B02512',
 'B02764',
 'B02617']

In [20]:
uberLinhas.filter(lambda linha: "B02617" in linha).count()

59

In [21]:
b02617_RDD = uberLinhas.filter(lambda linha: "B02617" in linha)

In [22]:
b02617_RDD.filter(lambda linha: int(linha[3]) > 15000).count()

6

In [23]:
b02617_RDD.filter(lambda linha: int(linha[3]) > 15000).collect()

[['B02617', '1/31/2015', '1394', '15756'],
 ['B02617', '2/6/2015', '1526', '15417'],
 ['B02617', '2/13/2015', '1590', '16996'],
 ['B02617', '2/14/2015', '1486', '16999'],
 ['B02617', '2/20/2015', '1574', '16856'],
 ['B02617', '2/21/2015', '1443', '16098']]

In [24]:
# operações de Map Reduce
# criando outro RDD e removendo a primeira linha (cabeçalho) > mapear e separar as colunas a partir da virgula
uberRDD2 = sc.textFile("uber.csv").filter(lambda line: "base" not in line).map(lambda line:line.split(","))

In [26]:
# redução por KEY
uberRDD2.map(lambda kp: (kp[0], int(kp[3])) ).reduceByKey(lambda k,v: k + v).collect()

[('B02765', 193670),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02512', 93786),
 ('B02764', 1914449),
 ('B02617', 725025)]

In [27]:
uberRDD2.map(lambda kp: (kp[0], int(kp[3])) ).reduceByKey(lambda k,v: k + v).takeOrdered(10,key = lambda x: -x[1])

[('B02764', 1914449),
 ('B02617', 725025),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02765', 193670),
 ('B02512', 93786)]